In [1]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
import asyncio
from typing import List, Optional
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [6]:
instructions = """
You are a News Summary Writer Agent. Your task is to take in a list of full news article contents and the user's preferences for news topics, styles, or focus, and produce a concise, accurate, and well-written news summary in Markdown format.

Follow these rules:

1. **Understand User Preferences**:
   - Carefully consider the user's stated interests, preferred topics, or areas of focus.
   - Prioritize including information from articles that match the user's preferences.
   - If an article is less relevant, include only key points that add context or value.

2. **Generate a Markdown News Summary**:
   - Summarize the actual content of each article, capturing key facts, findings, or events.
   - Use clear headings and subheadings if necessary to organize the summary.
   - Include bullet points or numbered lists for clarity when summarizing multiple points.
   - Maintain clean Markdown syntax throughout (e.g., `#`, `##`, `###`, `-`, `*`, etc.).

3. **Write in a Readable, Engaging Style**:
   - Ensure the text is coherent, smooth, and easy to read.
   - Avoid repeating content unnecessarily.
   - Highlight the most important and relevant aspects of the articles according to user preferences.

4. **Return Only the Markdown Summary**:
   - Do not include full raw articles, tool outputs, metadata, or internal instructions.
   - Only include the synthesized summary based on the article content.

5. **Respect Accuracy**:
   - Do not fabricate information or speculate beyond what the articles provide.
   - Ensure the summary accurately reflects the content of each article.

Always aim to create a concise, user-friendly news summary that uses the actual content of the articles while aligning with the user's interests.
"""


In [3]:
def create_user_prompt(articles: list[str], user_preferences: str) -> str:
    """
    Generates a user prompt for the News Digest Writer Agent.
    
    Parameters:
    - articles: List of strings, each containing the full text of a news article.
    - user_preferences: String describing the user's news interests and preferences.
    
    Returns:
    - A formatted prompt string for the agent.
    """
    
    # Join articles with clear separation
    formatted_articles = "\n\n---\n\n".join(articles)
    
    prompt = f"""
    You are given the following news articles:

    {formatted_articles}

    The user has expressed the following preferences for their news digest:

    {user_preferences}

    Please create a well-organized, engaging news digest in Markdown format based on these articles and the user's preferences. 
    - Summarize each article in 1-3 sentences.
    - Use headings, subheadings, and lists where appropriate.
    - Focus on articles most relevant to the user's preferences.
    - Do not include raw article text beyond summaries or any metadata.
    - Ensure the digest is readable, coherent, and professional.
    """
    return prompt


In [4]:
articles = ["""What you need to know

Google officially teased a few AI-powered features for the Pixel 10 on X via a short teaser clip.
The video seemingly teased the series' strong telephoto capabilities while also calling back to the Pixel 9's "Add Me."
Google confirms again that its launch event is set for August 20.
It's nearly go time, and Google is ramping up its teaser engine for the Pixel 10 series with some hints about Gemini.

This afternoon (Aug 15), Google posted a short teaser clip on X, briefly hinting at what consumers can expect out of the Pixel 10 series. Again, Google is sticking with its promotional tagline: "Ask more of your phone." The teaser runs through its first feature, which seems to touch on the phone's telephoto lens. Google asks, "What if the camera could get super, super, super close" to your subject, while you're undoubtedly pretty far away.

The teaser doesn't expand on that, but the clip rolls into quite a familiar feature that we've even seen tested during an I/O keynote.

The next question posed is: "What if you could be in the group photo even if you snapped the group photo?" This is, of course, leading us right into a pretty convenient Google Pixel feature that we first heard about in 2024. Aside from these two teasers—and the frame of a Pixel 10—Google's short clip doesn't give us much else to chew on.

The company has confirmed again that its event will take place on August 20.

Google in NYC

Additionally, Google started dropping some more concrete details about its August 20 event for new Pixels. The company states it will have a recognizable cast on set for the Pixels, such as the host of The Tonight Show, Jimmy Fallon. The Golden State Warriors' Stephen Curry, British racer Lando Curry, "and more" are teased to be at the Made by Google event.

The action goes down on August 20 at 1pm ET (10am PT).

It's digestible AI

This group photo feature Google's alluding to seems to almost certainly be "Add Me." In 2024, after the Made by Google event of that year, the Pixel 9 went live for pre-orders, but not before we saw Add Me showcased during the keynote. Android Central's Jerry Hildenbrand said it best: Add Me is "AI that people can understand." The Pixel 9 is capable of taking a group photo once, and then allowing the photographer to step in somewhere and snap another.

It then leverages its AI software to graciously merge those two photos to a point where it looks like someone else took the photo. Like a video game where it's just some camera floating by a string. While this feels like a feature we should've expected on the Pixel 10, it's nice to see Google (sort of) say so.

Let's also not forget about the foldable, the Pixel 10 Pro Fold, which has had some rumors surface quite recently. Those reports highlighted the existence of Qi2 support and a potential delay in its market release that consumers might need to keep in the back of their minds.

For more news and information on Google's upcoming phones, check out our Ultimate Guide.

Nickolas is always excited about tech and getting his hands on it. Writing for him can vary from delivering the latest tech story to scribbling in his journal. When Nickolas isn't hitting a story, he's often grinding away at a game or chilling with a book in his hand.""",
"""
Some people like to stay on the cutting edge of technology. If that's you, you already know that Google is going to officially unveil its Pixel 10 phones (including a new Fold) during its August 20th event. But if you're someone who prefers to use these opportunities to save on last-gen tech, you've been rewarded with a sweet deal on the Pixel 9 Pro Fold. The base 256GB version in obsidian (black) is down to $1,199 at Amazon and Best Buy, which is 33 percent off its original $1,799 price. It was selling for $100 more just last week.

The Pixel 9 Pro Fold was a big leap forward compared to Google's first-gen foldable, touting a lightweight design that looks and feels like a normal phone before you unfold it. One of its main drawbacks was that, despite its high original price, you don't get the same great cameras in the cheaper Pixel 9 Pro (which, by the way, is only $599 at Best Buy). They're still good, mind you, but reviewer Allison Johnson said during her testing that its telephoto lens produced softer images by comparison, and low-light photos were less detailed. Drawbacks aside, she was eager to recommend the foldable phone at its original $1,800 price.

The Pixel 10 Pro Fold is nearly upon us, and it'll likely deliver some sizeable improvements. Google itself has already shown off its design in YouTube clips, and it's very similar to the model on sale now. However, there's a rumor that it might have an IP68 rating, which would make it dustproof. Dust was the kryptonite for early foldable models, but the tech has apparently come a long way since then, so this would be a huge selling point in favor of the new model if it's true.

Other Verge-approved deals for your weekend

We had some tough (but warranted) words for the Windows version of the Lenovo Legion Go S gaming handheld in our review. The Verge's Sean Hollister summed it up by saying it “feels good, plays bad,” which I echo in my Polygon review from earlier this year. One of Sean's main complaints is the exorbitant $730 price that Lenovo was charging, especially since its specs aren't all that impressive compared to the Steam Deck, and don't get us started on the pain of using Windows 11 in handheld mode. However, its recent $300 off price drop at Best Buy to $499.99 makes it easier to overlook its faults. This model has AMD's Ryzen Z2 Go chip, 32GB of RAM, and a 1TB SSD. It's a decent value now considering that the version with SteamOS instead of Windows 11 costs $599.99, yet includes half the RAM and storage, though we'd still recommend picking a system with a Z1 Extreme chip. Also, note that Valve told us it doesn't officially support SteamOS on the Windows model.

If you're looking for your first — or next — Sonos speaker, there aren't too many deals worth flagging on new units right now. One good refurbished deal has come to the surface, though, and it's on the Sonos One SL, which is $119 at Wellbots for Verge readers. Enter the code SONOSVERGE40 at checkout to knock the price down from $159. This is an appealing deal because it beats the current prices at Sonos' own refurbished outlet, and because it comes with a one-year warranty through Sonos — just as new speakers do. The 2019 One SL can be used alone, or synced up with other Sonos Wi-Fi speakers through the company's app. It lacks a microphone, so you can't bark voice commands at it.

Those who own a 4K Blu-ray player, like the Panasonic DP-UB420-K, can add more flicks to their library of discs on the cheap over at Gruv. The shopping site (which is owned by Universal Pictures) is offering a bunch of 4K Blu-ray titles in a “3 for $30” sale, making each one just $10 before tax and shipping. Just to give you some examples of visually-stunning films that are eligible, there's Dunkirk, Crazy Rich Asians, The Matrix Resurrections, and more.
""",
]
user_preferences = "Show me news on AI and on what Google is up to. Also include any news about Trump and the economy."

In [5]:
prompt = create_user_prompt(articles=articles, user_preferences=user_preferences)

prompt

'\n    You are given the following news articles:\n\n    What you need to know\n\nGoogle officially teased a few AI-powered features for the Pixel 10 on X via a short teaser clip.\nThe video seemingly teased the series\' strong telephoto capabilities while also calling back to the Pixel 9\'s "Add Me."\nGoogle confirms again that its launch event is set for August 20.\nIt\'s nearly go time, and Google is ramping up its teaser engine for the Pixel 10 series with some hints about Gemini.\n\nThis afternoon (Aug 15), Google posted a short teaser clip on X, briefly hinting at what consumers can expect out of the Pixel 10 series. Again, Google is sticking with its promotional tagline: "Ask more of your phone." The teaser runs through its first feature, which seems to touch on the phone\'s telephoto lens. Google asks, "What if the camera could get super, super, super close" to your subject, while you\'re undoubtedly pretty far away.\n\nThe teaser doesn\'t expand on that, but the clip rolls int

In [7]:
news_agent = Agent(
    name="News Summarizer",
    instructions=instructions,
    model="gpt-4.1",
)

In [9]:
with trace("News Summary"):
    result = await Runner.run(news_agent, input=prompt)
    display(Markdown(result.final_output))

# News Digest

## Google & AI Updates

### Pixel 10 Series Teasers Highlight AI Features

- **Google has begun teasing new AI-powered features for its upcoming Pixel 10 smartphones**, emphasizing advances in telephoto camera capabilities and an improved "Add Me" group photo feature—an easy-to-use AI tool first seen with the Pixel 9, which allows the photographer to appear in group photos via image merging.
- The teaser video also hinted at Google's advancements in AI integration, particularly through its Gemini AI, with more details expected at the **Made by Google event on August 20** in New York City. The event will feature notable guests such as Jimmy Fallon, Stephen Curry, and Lando Norris.
- Rumors suggest the Pixel 10 Pro Fold may include upgrades like Qi2 wireless charging and possibly a dustproof IP68 rating, addressing earlier foldable device limitations.

### Pixel 9 Pro Fold Discounts

- As Google prepares for the Pixel 10 launch, the **Pixel 9 Pro Fold is now offered at a significant discount** (down 33% to $1,199), presenting a good value despite minor camera limitations compared to its non-foldable counterpart.
- The Pixel 10 Pro Fold is expected to build on its predecessor’s design and potentially resolve durability concerns, making it one to watch in the foldable phone market.

## Other Technology & Deal Highlights

- **Lenovo Legion Go S:** Now $499.99 (down $300), the Windows handheld gaming device gets an attractive price cut, though performance and Windows integration remain questionable.
- **Sonos One SL Refurbished:** Available for $119 with promo code, this speaker offers strong value for those seeking to expand audio setups without voice assistant features.
- **4K Blu-ray Movie Sale:** Select films available in a “3 for $30” offer through Gruv, appealing to physical media collectors.

---

## Trump & The Economy

*(No articles provided on Trump or the economy in this news cycle.)*

---

Stay tuned for more updates on Google’s AI advancements and future device launches as details emerge from their August 20 event.